In [3]:
!python3 -m pip install transformers
!python3 -m pip install torch



Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 26.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 50.6 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.3 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 KB 53.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to 

In [13]:
!python3 -m pip install scikit-learn scipy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 36.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 30.4 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 33.2 MB/s eta 0:00:00


In [37]:
!python3 -m pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 1.7 MB/s eta 0:00:00 MB/s eta 0:00:01


In [30]:

# ML
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

from torch.utils.data import Dataset, DataLoader
from torch import cuda

from transformers import AutoModel

# utils
import pandas as pd
# import seaborn as sns
import numpy as np
import tqdm

from sklearn.metrics import confusion_matrix


In [48]:
##fewer training data
train_path = './train.tsv'
test_path = './test.tsv'
##new training data with more samples
#train_path = './new_train.tsv'
#test_path = './new_test.tsv'
train_df = pd.read_csv(train_path, sep='\t')
test_df = pd.read_csv(test_path, sep='\t')
# train.text = train.text.apply(slp.rm_linebreaks).apply(slp.c2temp_2).apply(Tokenizer.tokenize_text).apply(lambda x: list(map(wlp.c2temp, x)))
# test.text = test.text.apply(slp.rm_linebreaks).apply(slp.c2temp_2).apply(Tokenizer.tokenize_text).apply(lambda x: list(map(wlp.c2temp, x)))
# train.head()
# train["text"][0]

In [49]:
from datasets import Dataset
train_text = train_df["text"].to_numpy() 
train_label = train_df["label"].to_numpy()

test_text = test_df["text"].to_numpy() 
test_label = test_df["label"].to_numpy()


train_data_dict = {"text":train_text, "label":train_label}
train_dataset = Dataset.from_dict(train_data_dict)


test_data_dict = {"text":test_text, "label":test_label}
test_dataset = Dataset.from_dict(test_data_dict)

In [50]:
from transformers import AutoTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
#choose the tokenizer that you would like to use

#tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')

tokenizer = AutoTokenizer.from_pretrained('m3rg-iitd/matscibert')

#trained on 2M articles to understand the specific language used in materials science articles 
#tokenizer = AutoTokenizer.from_pretrained('/home/defne/matbert-base-cased', do_lower_case=False)

#tokenizer = AutoTokenizer.from_pretrained('allenai/specter')



loading configuration file config.json from cache at /home/defne/.cache/huggingface/hub/models--m3rg-iitd--matscibert/snapshots/24a4e4318dda9bc18bff5e6a45debdcb3e1780e3/config.json
Model config BertConfig {
  "_name_or_path": "m3rg-iitd/matscibert",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file vocab.txt from cache at /home/defne/.cache/huggingface/hub/models--m3rg-iitd--matscibert/snapshots/24a4e4318dda9bc18bff5e6a45debdcb3e1780e3/vo

In [51]:
labels = ["action","constituent","unrelated","property"]
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

def transform_labels(label):
    label = label['label']
    if label == 'action':
        num = 0
    elif label == 'constituent':
        num = 1
    elif label == 'unrelated':
        num = 2
    elif label == 'property':
        num = 3
    else:
        num = -1

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example["text"], padding='max_length', max_length=512, truncation=True)


In [52]:
# tokenized_data = [tokenizer(each, padding='max_length', return_tensors="pt", max_length=512, truncation=True) for each in tqdm(train_text)]
train_dataset = train_dataset.map(tokenize_data, batched=True)
remove_columns = ['text','label']
train_dataset = train_dataset.map(transform_labels, remove_columns=remove_columns)

test_dataset = test_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(transform_labels, remove_columns=remove_columns)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/904 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/227 [00:00<?, ?ex/s]

In [53]:
train_dataset = train_dataset.shuffle(seed=10).select(range(len(train_text)))
eval_dataset = test_dataset.shuffle(seed=10).select(range(len(test_text)))

In [54]:
from transformers import TrainingArguments

# training_args = TrainingArguments("test_trainer", num_train_epochs=10)

args = TrainingArguments(
    "scibert",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
#     per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
#     per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [38]:
from datasets import load_metric

from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
import evaluate
#metric = load_metric("accuracy","f1")
#metric = evaluate.load("accuracy")
#metric = load_metric('accuracy')

#def compute_metrics(eval_pred):
#    predictions, labels = eval_pred
#    predictions = np.argmax(predictions, axis=1)
#    return  metric.compute(predictions=predictions, references=labels)

def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="weighted")
    precision = precision_score(y_true=labels, y_pred=pred, average="weighted")
    f1 = f1_score(y_true=labels, y_pred=pred, average="weighted")    
    return {"accuracy": accuracy, "precision": precision,"recall": recall, "f1": f1}

In [55]:
from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=4, label2id=label2id,id2label=id2label)
model = AutoModelForSequenceClassification.from_pretrained("m3rg-iitd/matscibert", num_labels=4, label2id=label2id,id2label=id2label)
# model = AutoModelForSequenceClassification.from_pretrained('/home/defne/matbert-base-cased', num_labels=4, label2id=label2id,id2label=id2label)
#model = AutoModelForSequenceClassification.from_pretrained("allenai/specter", num_labels=4, label2id=label2id,id2label=id2label)


loading configuration file config.json from cache at /home/defne/.cache/huggingface/hub/models--m3rg-iitd--matscibert/snapshots/24a4e4318dda9bc18bff5e6a45debdcb3e1780e3/config.json
Model config BertConfig {
  "_name_or_path": "m3rg-iitd/matscibert",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "action",
    "1": "constituent",
    "2": "unrelated",
    "3": "property"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "action": 0,
    "constituent": 1,
    "property": 3,
    "unrelated": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 

In [56]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=args, train_dataset=train_dataset, eval_dataset=eval_dataset, compute_metrics=compute_metrics
)

In [57]:
trainer.train()

/home/defne/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 904
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 280
  Number of trainable parameters = 109921540


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.821100,0.735551,0.753304,0.756647,0.753304,0.683432
1,0.271800,0.363916,0.854626,0.839461,0.854626,0.841359
2,0.230900,0.320322,0.876652,0.872293,0.876652,0.872778
3,0.116800,0.320159,0.889868,0.891134,0.889868,0.884189
4,0.161500,0.379134,0.911894,0.912555,0.911894,0.912147
5,0.172100,0.371230,0.911894,0.914274,0.911894,0.912589
6,0.100100,0.404912,0.911894,0.911868,0.911894,0.908916
7,0.055400,0.440877,0.903084,0.902673,0.903084,0.901082
8,0.002100,0.457030,0.903084,0.902673,0.903084,0.901082
9,0.002500,0.456859,0.903084,0.903703,0.903084,0.900984


***** Running Evaluation *****
  Num examples = 227
  Batch size = 8
/home/defne/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to scibert/checkpoint-28
Configuration saved in scibert/checkpoint-28/config.json
Model weights saved in scibert/checkpoint-28/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 227
  Batch size = 8
Saving model checkpoint to scibert/checkpoint-56
Configuration saved in scibert/checkpoint-56/config.json
Model weights saved in scibert/checkpoint-56/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 227
  Batch size = 8
Saving model checkpoint to scibert/checkpoint-84
Configuration saved in scibert/checkpoint-84/config.json
Model weights saved in scibert/checkpoint-

TrainOutput(global_step=280, training_loss=0.22112641689808307, metrics={'train_runtime': 910.1922, 'train_samples_per_second': 9.932, 'train_steps_per_second': 0.308, 'total_flos': 2376461725827072.0, 'train_loss': 0.22112641689808307, 'epoch': 9.99})

In [58]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 227
  Batch size = 8


{'eval_loss': 0.3712295889854431,
 'eval_accuracy': 0.9118942731277533,
 'eval_precision': 0.9142739609043855,
 'eval_recall': 0.9118942731277533,
 'eval_f1': 0.9125891915368864,
 'eval_runtime': 7.7321,
 'eval_samples_per_second': 29.358,
 'eval_steps_per_second': 3.751,
 'epoch': 9.99}